In [1]:
# Install the necessary libraries and clone the repository
!pip install ninja
!pip install torch torchvision
!pip install pillow
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import os
import torch
import pickle
import PIL.Image
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import math

In [3]:
# Moving to the stylegan2-ada-pytorch directory
os.chdir('stylegan2-ada-pytorch')

In [4]:
# Create a models folder and download the pretreated model
!mkdir models
!wget -O models/ffhq.pkl https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl

--2024-06-08 08:33:31--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 3.163.158.113, 3.163.158.83, 3.163.158.34, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|3.163.158.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381646055 (364M) [binary/octet-stream]
Saving to: ‘models/ffhq.pkl’

models/ffhq.pkl     100%[===================>] 363.97M  25.1MB/s    in 16s     

2024-06-08 08:33:48 (22.6 MB/s) - ‘models/ffhq.pkl’ saved [381646055/381646055]



In [5]:
# Checking the current directory. All generated images will be saved in this location.
!pwd
! mkdir -p ../images

/content/stylegan2-ada-pytorch


In [6]:
import dnnlib
import legacy

# Generating images of ten people
def load_generator(model_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    with open(model_path, 'rb') as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device)
    return G

def generate_image(generator, z, truncation_psi=0.5, noise_mode='const'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    img = generator(z, None, truncation_psi=truncation_psi, noise_mode=noise_mode)
    img = img[0].permute(1, 2, 0).cpu().numpy()
    img = (img * 127.5 + 128).clip(0, 255).astype(np.uint8)
    return PIL.Image.fromarray(img, 'RGB')

def generate_variations(generator, seed, num_variations=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.manual_seed(seed)
    rnd = np.random.RandomState(seed)

    z = torch.tensor(rnd.randn(1, generator.z_dim), device=device)
    images = [generate_image(generator, z)]

    for i in range(1, num_variations + 1):
        variation_z = z + torch.tensor(rnd.randn(1, generator.z_dim) * 0.1 * i, device=device)
        noise_mode = 'const' if i % 2 == 0 else 'random'
        images.append(generate_image(generator, variation_z, noise_mode=noise_mode))

    return images

model_path = 'models/ffhq.pkl'
generator = load_generator(model_path)
seeds = [109, 10, 70, 30, 35, 38, 40, 100, 116, 115]

output_dir = '../images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for image_idx, seed in enumerate(seeds):
  person_directory = f'{output_dir}/person_{image_idx}'
  if not os.path.exists(person_directory):
    os.makedirs(person_directory)

  images = generate_variations(generator, seed)
  for idx, img in enumerate(images):
    img.save(f'{person_directory}/generated_face_{seed}_variation_{idx}.png')

Setting up PyTorch plugin "bias_act_plugin"... 

/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... 

/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.


In [7]:
%cd ../images

/content/images


In [9]:
# Add watermarks informing that images are generated by AI.
# It is necessary to add the Arial.ttf file posted in the repository to the /content/stylegan2-ada-pytorch directory.
# Click the folder icon on the left and add the file to the folder named stylegan2-ada-pytorch.

def add_watermark(image_path, person_idx, seed):
    # Load the image
    image = Image.open(image_path)

    # Set the watermark text and its style
    watermark_text = "generated by AI"
    font_size = 120
    font = ImageFont.truetype("../Arial.ttf", font_size)

    # Calculate the diagonal length of the image
    diagonal_length = math.sqrt(image.width ** 2 + image.height ** 2)

    # Calculate the width and height of the text
    text_width, text_height = font.getsize(watermark_text)

    # Calculate the new font size based on the diagonal length of the image
    new_font_size = int(font_size * diagonal_length / (image.width + image.height))
    font = ImageFont.truetype("../Arial.ttf", new_font_size)

    # Calculate the position of the text along the diagonal of the image
    x_offset = (image.width - text_width) // 2
    y_offset = (image.height - text_height) // 2

    # Create a new layer with transparency
    watermark = Image.new("RGBA", (image.width, image.height), (255, 255, 255, 0))
    draw = ImageDraw.Draw(watermark)

    # Draw the text on the layer and rotate it by 45 degrees
    draw.text((x_offset, y_offset), watermark_text, font=font, fill=(255, 255, 255, 128))
    watermark_rotated = watermark.rotate(45, expand=True)

    # Resize the watermark image to fit the main image
    watermark_resized = watermark_rotated.resize(image.size, Image.ANTIALIAS)

    # Composite the watermark layer over the original image
    watermarked_image = Image.alpha_composite(image.convert("RGBA"), watermark_resized)

    # Save the image with the watermark
    watermarked_image.save(f'person_{person_idx}/watermarked_generated_face_{seed}.png')

# Adding a watermark to all previously generated images. Images with watermarks are also saved in current directory.

seeds = [109, 10, 70, 30, 35, 38, 40, 100, 116, 115]
for person_idx, seed in enumerate(seeds):
  add_watermark(f'person_{person_idx}/generated_face_{seed}_variation_0.png', person_idx, seed)

<ipython-input-9-6a1b60e6d937>:18: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_width, text_height = font.getsize(watermark_text)
<ipython-input-9-6a1b60e6d937>:37: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  watermark_resized = watermark_rotated.resize(image.size, Image.ANTIALIAS)


## Generating documents

In [10]:
!pwd

/content/images


In [11]:
class Person():
  def __init__(self, id, name, class_name, phone, address, image_path):
    self.id = id
    self.name = name
    self.class_name = class_name
    self.phone = phone
    self.address = address
    self.image_path = image_path

In [12]:
people = [
    Person("89302745162", "Emily Smith", "1A", "555 444 333", "Smith Street, New York", "person_0/generated_face_109_variation_0.png"),
    Person("98701234567", "Emily Johnson", "1B", "123456789", "Johnson Street, New York", "person_1/generated_face_10_variation_0.png"),
    Person("24680135790", "Richard Johnson", "3A", "(999) 888-7777", "Johnson Street, New York", "person_2/generated_face_70_variation_0.png"),
    Person("67439105827", "Michael Thompson", "2B", "(555) 234-5678", "Thompson Street, New York", "person_3/generated_face_30_variation_0.png"),
    Person("89123456789", "Minh Nguyen", "2D", "(12) 456-7890", "Nguyen Street, Californie", "person_4/generated_face_35_variation_0.png"),
    Person("12345678901", "Jenny Li", "2A", "(888) 555-1234", "Li Street, New York", "person_5/generated_face_38_variation_0.png"),
    Person("9865432109", "Jason Chen", "2D", "(777) 222-3333", "Chen Street, Washington", "person_6/generated_face_40_variation_0.png"),
    Person("13579246801", "Priya Patel", "1D", "(666) 777-8888", "Patel Street, New York", "person_7/generated_face_100_variation_0.png"),
    Person("11223344556", "Theo Demetriou", "1A", "(333) 666-5555", "Demetriou Street, New York", "person_8/generated_face_116_variation_0.png"),
    Person("54321098765", "Amira Khan", "2C", "(444) 555-6666", "Khan Street, New York", "person_9/generated_face_115_variation_0.png"),
]

In [16]:
def generate_library_card(person: Person):
  image_path = '../document_schemas/Library_card.png'
  image = Image.open(image_path)

  draw = ImageDraw.Draw(image)
  font_path = '../Arial.ttf'
  font_size = 22
  font = ImageFont.truetype(font_path, font_size)

  # Text values
  text_position = [600, 252]

  for person_information in [person.name, person.id, person.class_name, person.phone, person.address]:
    draw.text(text_position, person_information, font=font, fill=(0, 0, 0))
    text_position[1] += 40

  # Image
  overlay_path = person.image_path
  overlay = Image.open(overlay_path).convert("RGBA")
  desired_size = (250, 320)

  overlay.thumbnail(desired_size, Image.LANCZOS)
  overlay_position = (140, 230)
  image.paste(overlay, overlay_position, overlay)

  # Save
  dir = person.image_path.split('/')[0]
  image.save(f'{dir}/university_card.png')

In [39]:
def generate_bicycle_card(person: Person):
  image_path = '../document_schemas/Bicycle_card.png'
  image = Image.open(image_path)

  draw = ImageDraw.Draw(image)
  font_path = '../Arial.ttf'
  font_size = 22
  font = ImageFont.truetype(font_path, font_size)
  text_position = [500, 220]

  # Text data
  draw.text(text_position, person.name, font=ImageFont.truetype(font_path, 40), fill=(0, 0, 0))

  text_position[1] += 80
  draw.text(text_position, f'ID Number: {person.id}', font=font, fill=(120, 120, 120))
  text_position[1] += 40
  draw.text(text_position, f'Phone Number: {person.phone}', font=font, fill=(120, 120, 120))
  text_position[1] += 40
  draw.text(text_position, f'Address: {person.address}', font=font, fill=(120, 120, 120))


  # Image
  overlay_path = person.image_path
  overlay = Image.open(overlay_path).convert("RGBA")
  desired_size = (254, 253)

  overlay.thumbnail(desired_size, Image.LANCZOS)
  mask = Image.new("L", overlay.size, 0)
  draw = ImageDraw.Draw(mask)
  draw.ellipse((0, 0, overlay.size[0], overlay.size[1]), fill=255)

  circular_overlay = Image.new("RGBA", overlay.size)
  circular_overlay.paste(overlay, mask=mask)

  if circular_overlay.size != desired_size:
      circular_overlay = circular_overlay.resize(desired_size, Image.LANCZOS)

  overlay_position = (130, 200)
  image.paste(circular_overlay, overlay_position, circular_overlay)

  # Save
  dir = person.image_path.split('/')[0]
  image.save(f'{dir}/bicycle_card.png')

In [40]:
for person in people:
  generate_library_card(person)
  generate_bicycle_card(person)